In [1]:
from collections import Counter

In [2]:
import re
import bs4 as bs
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import json

## company cleanup

In [7]:
comp = pd.read_csv('angelist_unique_4261.csv')

In [8]:
comp.drop(["Unnamed: 0"],axis=1,inplace=True)
comp.dropna(how="all",inplace=True)

In [9]:
Counter([str(i)[0] for i in comp['raised']])

Counter({'$': 4202, '€': 35, '£': 23})

In [10]:
def clean_currency(curr_string):
    curr_string=str(curr_string)
    curr_string = curr_string.replace(",","")
    curr_type = curr_string[0]
    curr_val = int(curr_string[1:])
    
    if curr_type == "$":
        return int(curr_val)
    elif curr_type == "€":
        return int(curr_val*1.11)
    elif curr_type == "£":
        return int(curr_val*1.31)
    else: return int(curr_val)
    

In [11]:
clean_currency("€904,500")

1003995

In [12]:
comp['raised'] = comp['raised'].apply(lambda row: clean_currency(row))

In [13]:
comp.sort_values(by=['raised'])

,joined,kind,link,location,market,name,pitch,raised,size,stage,website
598,Oct ’13,Startup,https://angel.co/company/chartos-1,New York City,Mobile Health,Chartos,\nBetter health information. Better health.\n,1000,1-10,Seed,chartos.com
845,Feb ’13,Startup,https://angel.co/company/dtn-electronics,New York City,Retail Technology,DTN Electronics,\nOff-price retail/wholesale ecommerce store\n,1000,1-10,Seed,dtnelectronics.com
2925,Jan ’15,Startup,https://angel.co/company/sracy,Ajmer,Online Gaming,SRACY,\nNetflix for Software or Games!\n,1200,1-10,NaN,sracy.com
1698,Nov ’14,Startup,https://angel.co/company/intellic,Bengaluru,Web Development,Intellic,\nflickr\n,1556,11-50,Series A,intellic.co
1483,Mar ’15,Startup,https://angel.co/company/hgc,China,Gambling,HGC,\nHand Gesture Software \n,2000,1-10,NaN,handcontrolstudio.sinaapp.com
...,...,...,...,...,...,...,...,...,...,...,...
3121,May ’14,Startup,https://angel.co/company/snapchat,Los Angeles,Video Streaming,Snapchat,"\nIt's about the moment, a connection between ...",3819085000,51-200,NaN,snap.com
2023,Jul ’13,Startup,https://angel.co/company/lyft,Palo Alto,Collaborative Consumption,Lyft,\nImproving people’s lives with the world’s be...,5312500000,5000+,NaN,lyft.com
3257,Oct ’16,Startup,https://angel.co/company/stemcentrx,-,-,StemCentRx,\nBiotechnology Company\n,10473500000,-,Series F,stemcentrx.com
3622,Jun ’10,Startup,https://angel.co/company/uber,San Francisco,Technology,Uber,\nThe better way to get there\n,12252450000,5000+,NaN,uber.com


In [14]:
comp['size'].unique().tolist()

['11-50',
 '1-10',
 '51-200',
 '-',
 '201-500',
 nan,
 '1001-5000',
 '501-1000',
 '5000+']

In [15]:
comp.dtypes

joined      object
kind        object
link        object
location    object
market      object
name        object
pitch       object
raised       int64
size        object
stage       object
website     object
dtype: object

In [16]:
import datetime

x = datetime.datetime(2020, 5, 1)

print(x)

2020-05-01 00:00:00


In [17]:
m={'Jan':1,'Feb':2,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12}
def clean_date(curr_date):
    mo,year = (curr_date.split(' ’'))
    return(datetime.datetime(int('20'+year), m[mo], 1))


In [18]:
clean_date('Jun ’10')

datetime.datetime(2010, 6, 1, 0, 0)

In [19]:
def year_estd(date):
    now = datetime.datetime(2019,12,15,0,0,0,0)
    diff = now - date
    return (diff.days/365)

In [20]:
year_estd(clean_date('Dec ’15'))

4.041095890410959

In [21]:
comp['joined'] = comp['joined'].apply(lambda row: clean_date(row))

In [22]:
comp.head()

,joined,kind,link,location,market,name,pitch,raised,size,stage,website
0,2013-10-01,Startup,https://angel.co/company/urx,San Francisco,Mobile Commerce,NaN,"\nThe Deeplink Search Engine, For Developers (...",15100000,11-50,Acquired,urx.com
1,2012-05-01,Startup,https://angel.co/company/ginx,Las Vegas,Mobile Commerce,*Ginx,\nImage based marketing and analytics \n,700000,1-10,Seed,ginxapp.com
2,2011-09-01,Startup,https://angel.co/company/1-page,San Francisco,SaaS,1-Page,\nNext Generation Enterprise Recruitment Proce...,163000000,11-50,IPO,1-page.com
3,2012-04-01,Startup,https://angel.co/company/121nexus,Cambridge,Politics,121nexus,\nThe information platform for physical things.\n,794000,1-10,-,121nexus.com
4,2013-11-01,Startup,https://angel.co/company/1234enter,São Paulo,Small and Medium Businesses,1234ENTER,\nAutomated digital marketing guidance for Sma...,650267,1-10,NaN,1234enter.com.br


In [23]:
comp['years'] = comp['joined'].apply(lambda row: year_estd(row))

In [24]:
comp['annual_raised'] = comp['raised']/comp['years']

In [25]:
comp['raised_2mil'] = comp['raised'].apply(lambda row: 1 if row>2000000 else 0)

In [26]:
comp['raised_12mil'] = comp['raised'].apply(lambda row: 1 if row>12000000 else 0)

In [27]:
size_dic={}
size_dic['1-10'] = 5
size_dic['11-50'] = 25
size_dic['51-200'] = 125
size_dic['201-500'] = 350
size_dic['501-1000'] = 750
size_dic['1001-5000'] = 3000
size_dic['5000+'] = 8000
size_dic['-'] = np.nan
size_dic[np.nan] = np.nan
size_dic

{'1-10': 5,
 '11-50': 25,
 '51-200': 125,
 '201-500': 350,
 '501-1000': 750,
 '1001-5000': 3000,
 '5000+': 8000,
 '-': nan,
 nan: nan}

In [28]:
comp['size_numeric'] = comp['size'].apply(lambda row: size_dic[row])

In [29]:
comp

,joined,kind,link,location,market,name,pitch,raised,size,stage,website,years,annual_raised,raised_2mil,raised_12mil,size_numeric
0,2013-10-01,Startup,https://angel.co/company/urx,San Francisco,Mobile Commerce,NaN,"\nThe Deeplink Search Engine, For Developers (...",15100000,11-50,Acquired,urx.com,6.208219,2.432259e+06,1,1,25.0
1,2012-05-01,Startup,https://angel.co/company/ginx,Las Vegas,Mobile Commerce,*Ginx,\nImage based marketing and analytics \n,700000,1-10,Seed,ginxapp.com,7.627397,9.177443e+04,0,0,5.0
2,2011-09-01,Startup,https://angel.co/company/1-page,San Francisco,SaaS,1-Page,\nNext Generation Enterprise Recruitment Proce...,163000000,11-50,IPO,1-page.com,8.293151,1.965477e+07,1,1,25.0
3,2012-04-01,Startup,https://angel.co/company/121nexus,Cambridge,Politics,121nexus,\nThe information platform for physical things.\n,794000,1-10,-,121nexus.com,7.709589,1.029886e+05,0,0,5.0
4,2013-11-01,Startup,https://angel.co/company/1234enter,São Paulo,Small and Medium Businesses,1234ENTER,\nAutomated digital marketing guidance for Sma...,650267,1-10,NaN,1234enter.com.br,6.123288,1.061957e+05,0,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4255,2012-10-01,Startup,https://angel.co/company/zoomsquare,Berlin,Lead Generation,zoomsquare,\nDie Partnerschaftsvermittlung auf Quadratmet...,540000,11-50,Seed,zoomsquare.com,7.208219,7.491448e+04,0,0,25.0
4256,2014-02-01,Startup,https://angel.co/company/zoovu,London,Point of Sale,zoovu,\nzoovu is an engagement platform that helps ...,6000000,51-200,Series A,zoovu.com,5.871233,1.021932e+06,1,0,125.0
4257,2012-03-01,Startup,https://angel.co/company/entrobit-agora,Moscow,Wholesale,СentroBit Agora,\nMultichannel e-commerce platform for SMB mar...,450000,11-50,Seed,centrobit.ru,7.794521,5.773286e+04,0,0,25.0
4258,2013-12-01,Startup,https://angel.co/company/shwrum,Gainesville,Small and Medium Businesses,​Shwrüm,\nMobile CRM for Boutiques\n,50000,1-10,-,shwrum.com,6.041096,8.276644e+03,0,0,5.0


In [30]:
comp.to_csv('companies.csv')